In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import cv2

In [2]:
train_data_names = pd.read_csv('../data/df_final_image_data.csv')
train_data_names['Date'] = pd.to_datetime(train_data_names['Date'])
from os import listdir
from os.path import isfile, join
my_path = 'D:\\Indas\\Preprocessed\\Preprocessed\\NOK_samples_xray\\004'

In [24]:
train_data_names['Date'].drop_duplicates()[0] ==  train_data_names['Date'] 

0       True
1       True
2       True
3       True
4       True
       ...  
150    False
151    False
152    False
153    False
154    False
Name: Date, Length: 155, dtype: bool

In [3]:
idx_0206 = (train_data_names['Date'] == dt.datetime(2022, 2, 6) )
idx_OK = train_data_names.group == 'OK'

In [4]:
img_arr_02_06_OK = np.array([
    cv2.imread( my_path + '\\' + img_name, cv2.IMREAD_GRAYSCALE) for  img_name in train_data_names.name[np.logical_and(idx_0206, idx_OK)].values
    ])
img_arr_02_06_NOK = np.array(
    [cv2.imread( my_path + '\\' + img_name, cv2.IMREAD_GRAYSCALE) for  img_name in train_data_names.name[np.logical_and(idx_0206, idx_OK.__invert__())].values
     ])

In [11]:
test_info = train_data_names[np.logical_and(idx_0206, idx_OK.__invert__())]

In [69]:
exec(open('../rawCode/Reconstructer_Classes.py').read())
exec(open('../rawCode/HelperFunctions.py').read())
exec(open('../rawCode/Filter_Classes.py').read())

In [24]:
def MapDetectorPipeline(type, sg, nc, th, std_th, errG_op, errG_cl, errG_threshold, img_array_train, img_array_test):
    Reconstructer = pcaRecon(type, sg, nc)
    Reconstructer.fit(img_array_train)
    TrainReconstructions = np.array([Reconstructer.reconstruct(img_array_train[i,:,:]) for i in range(img_array_train.shape[0])])
    THFilter = threshold_filter(th)
    StdFilter = std_filter(std_th, 2)
    StdFilter.fit(TrainReconstructions, img_array_train)
    EgFilter = errGrowth_filter(errG_op, errG_cl, errG_threshold)
    mFilter = morphological_filter(1)
    del img_array_train
    del TrainReconstructions
    ErrTestReconstructions = [Reconstructer.reconstruct(img_array_test[i,:,:]) - img_array_test[i,:,:]  for i in range(img_array_test.shape[0])]
    th_filtered_TestReconstructions = [THFilter.filter(recon) for recon in ErrTestReconstructions]
    std_filtered_TestReconstructions = [StdFilter.filter(recon) for recon in th_filtered_TestReconstructions]
    eg_filtered_TestReconstructions = [EgFilter.filter(recon) for recon in std_filtered_TestReconstructions]
    del ErrTestReconstructions
    del th_filtered_TestReconstructions
    return([[mFilter.filter(std_fil) for std_fil in  std_filtered_TestReconstructions], eg_filtered_TestReconstructions] )

In [8]:
def evaluateErrorList(errMap, df):
    res_bool = []
    no_cl = []
    for i in range(len(errMap)):
        r_min, r_max,c_min, c_max = df.iloc[i][['row_min', 'row_max', 'col_min', 'col_max']]
        b, cl = evaluate_error_map(errMap[i] > 0, r_min, r_max, c_min, c_max)
        res_bool.append(b)
        no_cl.append(cl) 
    return(pd.DataFrame({'error_included': res_bool, 'number_of_clusters': no_cl}))

In [25]:
row_std, row_eG = MapDetectorPipeline('row', sg = 144, nc = 65, th =15, std_th = 3, errG_op = 1, errG_cl =3, errG_threshold = 5, img_array_train = img_arr_02_06_OK, img_array_test = img_arr_02_06_NOK )

In [27]:
evaluateErrorList(row_std, test_info).mean()

error_included          0.916667
number_of_clusters    901.500000
dtype: float64

In [26]:
evaluateErrorList(row_eG, test_info).mean()

error_included          0.916667
number_of_clusters    287.916667
dtype: float64

In [29]:
col_std, col_eG = MapDetectorPipeline('col', sg = 96, nc = 35, th =15, std_th = 3, errG_op =1, errG_cl =3, errG_threshold = 5, img_array_train = img_arr_02_06_OK, img_array_test = img_arr_02_06_NOK )

In [30]:
evaluateErrorList(col_std, test_info).mean()

error_included          0.916667
number_of_clusters    885.791667
dtype: float64

In [31]:
evaluateErrorList(col_eG, test_info).mean()

error_included          0.916667
number_of_clusters    280.083333
dtype: float64

In [44]:
sub_std, sub_eG = MapDetectorPipeline('sub', sg = 24, nc = 25, th =15, std_th = 3, errG_op =1, errG_cl =3, errG_threshold = 5, img_array_train = img_arr_02_06_OK, img_array_test = img_arr_02_06_NOK )

In [45]:
evaluateErrorList(sub_std, test_info).mean()

error_included          0.958333
number_of_clusters    630.416667
dtype: float64

In [46]:
evaluateErrorList(sub_eG, test_info).mean()

error_included          0.875000
number_of_clusters    284.583333
dtype: float64

In [47]:
aug_mer_fil = augmented_merge_filter(3)

In [48]:
evaluateErrorList([aug_mer_fil.filter_merge([sub_std[i], row_std[i], col_std[i]]) for i in range(len(sub_std))], test_info).mean()

error_included         0.666667
number_of_clusters    99.375000
dtype: float64

In [50]:
evaluateErrorList([aug_mer_fil.filter_merge([sub_eG[i], row_eG[i], col_eG[i]]) for i in range(len(sub_std))], test_info)

,error_included,number_of_clusters
0,True,16
1,False,15
2,True,16
3,False,8
4,True,15
5,True,17
6,True,10
7,True,69
8,True,37
9,True,27


In [52]:
del sub_std, sub_eG, col_std, col_eG, row_std, row_eG 

In [54]:
def MapDetectorPipelineQuantiles(type, sg, nc, th, errG_op, errG_cl, errG_threshold, img_array_train, img_array_test):
    Reconstructer = pcaRecon(type, sg, nc)
    Reconstructer.fit(img_array_train)
    TrainReconstructions = np.array([Reconstructer.reconstruct(img_array_train[i,:,:]) for i in range(img_array_train.shape[0])])
    THFilter = threshold_filter(th)
    q95Filter = quantile_filter(.95)
    q95Filter.fit(TrainReconstructions, img_array_train)
    EgFilter = errGrowth_filter(errG_op, errG_cl, errG_threshold)
    mFilter = morphological_filter(1)
    del img_array_train
    del TrainReconstructions
    ErrTestReconstructions = [Reconstructer.reconstruct(img_array_test[i,:,:]) - img_array_test[i,:,:]  for i in range(img_array_test.shape[0])]
    th_filtered_TestReconstructions = [THFilter.filter(recon) for recon in ErrTestReconstructions]
    q95_filtered_TestReconstructions = [q95Filter.filter(recon) for recon in th_filtered_TestReconstructions]
    eg_filtered_TestReconstructions = [EgFilter.filter(recon) for recon in q95_filtered_TestReconstructions]
    del ErrTestReconstructions
    del th_filtered_TestReconstructions
    return([[mFilter.filter(std_fil) for std_fil in  q95_filtered_TestReconstructions], eg_filtered_TestReconstructions] )

In [55]:
row_std, row_eG = MapDetectorPipelineQuantiles('row', sg = 144, nc = 65, th =15, errG_op=2, errG_cl =3, errG_threshold = 5, img_array_train = img_arr_02_06_OK, img_array_test = img_arr_02_06_NOK )
col_std, col_eG = MapDetectorPipelineQuantiles('col', sg = 96, nc = 35, th =15, errG_op=2, errG_cl =3, errG_threshold = 5, img_array_train = img_arr_02_06_OK, img_array_test = img_arr_02_06_NOK )
sub_std, sub_eG = MapDetectorPipelineQuantiles('sub', sg = 24, nc = 25, th =15, errG_op=2, errG_cl =3, errG_threshold = 5, img_array_train = img_arr_02_06_OK, img_array_test = img_arr_02_06_NOK )

In [87]:
evaluateErrorList(row_eG, test_info).assign(Methods= ['rowEg'] * len(row_eG), Date = pd.to_datetime(test_info.Date.values) )

,error_included,number_of_clusters,Methods,Date
0,True,51,rowEg,2022-02-06
1,True,45,rowEg,2022-02-06
2,True,76,rowEg,2022-02-06
3,False,59,rowEg,2022-02-06
4,True,60,rowEg,2022-02-06
5,True,63,rowEg,2022-02-06
6,True,61,rowEg,2022-02-06
7,True,128,rowEg,2022-02-06
8,False,112,rowEg,2022-02-06
9,True,110,rowEg,2022-02-06


In [60]:
evaluateErrorList(col_eG, test_info).mean()

error_included          0.833333
number_of_clusters    107.541667
dtype: float64

In [59]:
evaluateErrorList(sub_eG, test_info).mean()

error_included         0.833333
number_of_clusters    97.833333
dtype: float64

In [71]:
aug_mer_fil_maj = augmented_merge_majority_filter(3)
evaluateErrorList([aug_mer_fil_maj.filter_merge([sub_eG[i], row_eG[i], col_eG[i]]) for i in range(len(sub_std))], test_info).mean()

error_included         0.833333
number_of_clusters    32.416667
dtype: float64

In [72]:
evaluateErrorList([aug_mer_fil.filter_merge([sub_eG[i], row_eG[i], col_eG[i]]) for i in range(len(sub_std))], test_info).mean()

error_included        0.625000
number_of_clusters    3.958333
dtype: float64